<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# HugeCTR Python Interface

## Overview

In HugeCTR version 3.1, we provide an enhanced Python inferface, which supports continuous training and inference with high-level APIs. There are three main improvements. Firstly, the model graph can be constructed and dumped to the JSON file with Python code and it saves users from writing JSON configuration files. Secondly, we support the feature of model oversubscription with high-level APIs and extend it further for online training cases. Thirdly, the freezing method is provided for both sparse embedding and dense network, which enables transfer learning and fine-tune for CTR tasks.

This notebook explains how to access and use the enhanced HugeCTR Python interface. Please NOTE that the low-level training APIs are still maintained for users who want to have precise control of each training iteration, while migrating to the high-level training APIs is strongly recommended. For more details of the usage of Python API, please refer to [HugeCTR Python Interface](../docs/python_interface.md).

## Table of Contents
-  [Access the HugeCTR Python Interface](#1)
-  [DCN Demo](#2)
   * [Download and Preprocess Data](#21)
   * [Train from Scratch](#22)
   * [Continue Training](#23)
   * [Inference](#24)
-  [Wide&Deep Demo](#3)
   * [Download and Preprocess Data](#31)
   * [Model Oversubscription](#32)
   * [Fine-tune](#33)
   * [Low-level Training](#34)

<a id="1"></a>
## 1. Access the HugeCTR Python Interface

1. Please make sure that you have started the notebook inside the running NGC docker container: `nvcr.io/nvidia/merlin/merlin-inference:0.4`.

   A dynamic link to the `hugectr.so` library is installed to the system path `/usr/local/hugectr/lib/`. Besides, this system path is added to the environment variable `PYTHONPATH`, which means that you can use the Python interface within the docker container environment. Check the dynamic link with the following command:

In [ ]:
!ls /usr/local/hugectr/lib

2. Import HugeCTR, in order to train your model with Python as shown here:

In [1]:
import hugectr

<a id="2"></a>
## 2. DCN Demo

<a id="21"></a>
### 2.1 Download and Preprocess Data
1. Download the Kaggle Criteo dataset using the following command:
   ```shell
   $ cd ${project_root}/tools
   $ wget http://azuremlsampleexperiments.blob.core.windows.net/criteo/day_1.gz
   ```
   
   In preprocessing, we will further reduce the amounts of data to speedup the preprocessing, fill missing values, remove the feature values whose occurrences are very rare, etc. Here we choose pandas preprocessing method to make the dataset ready for HugeCTR training.

2. Preprocessing by Pandas using the following command:
   ```shell
   $ bash preprocess.sh 1 dcn_data pandas 1 0
   ```
   
   The first argument represents the dataset postfix. It is 1 here since day_1 is used. The second argument dcn_data is where the preprocessed data is stored. The fourth arguement (one after pandas) 1 embodies that the normalization is applied to dense features. The last argument 0 means that the feature crossing is not applied.

3. Create a soft link to the dataset folder using the following command:
   ```shell
   $ ln ${project_root}/tools/dcn_data ${project_root}/notebooks/dcn_data
   ```

<a id="22"></a>
### 2.2 Train from Scratch

We can train fom scratch, dump the model graph to a JSON file, and save the model weights and optimizer states by doing the following with Python APIs:

1. Create the solver, reader and optimizer, then initialize the model.
2. Construct the model graph by adding input, sparse embedding and dense layers in order.
3. Compile the model and have an overview of the model graph.
4. Dump the model graph to the JSON file.
5. Fit the model, save the model weights and optimizer states implicitly.

Please note that the training mode is determined by `repeat_dataset` within `hugectr.CreateSolver`. If it is True, the non-epoch mode training will be adopted and the maximum iterations should be specified by `max_iter` within `hugectr.Model.fit`. If it is False, then the epoch-mode training will be adopted and the number of epochs should be specified by `num_epochs` within `hugectr.Model.fit`.

The optimizer that is used to initialize the model applies to the weights of dense layers, while the optimizer for each sparse embedding layer can be specified independently within `hugectr.SparseEmbedding`.

In [2]:
%%writefile dcn_train.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 1500,
                              batchsize_eval = 4096,
                              batchsize = 4096,
                              i64_input_key = False,
                              use_mixed_precision = False,
                              repeat_dataset = True,
                              use_cuda_graph = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                                  source = ["./dcn_data/file_list.txt"],
                                  eval_source = "./dcn_data/file_list_test.txt",
                                  check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
model = hugectr.Model(solver, reader, optimizer)
model.add(hugectr.Input(label_dim = 1, label_name = "label",
                        dense_dim = 13, dense_name = "dense",
                        data_reader_sparse_param_array = 
                        [hugectr.DataReaderSparseParam(hugectr.DataReaderSparse_t.Distributed, 30, 1, 26)],
                        sparse_names = ["data1"]))
model.add(hugectr.SparseEmbedding(embedding_type = hugectr.Embedding_t.DistributedSlotSparseEmbeddingHash, 
                            max_vocabulary_size_per_gpu = 1447751,
                            embedding_vec_size = 16,
                            combiner = 0,
                            sparse_embedding_name = "sparse_embedding1",
                            bottom_name = "data1",
                            optimizer = optimizer))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Reshape,
                            bottom_names = ["sparse_embedding1"],
                            top_names = ["reshape1"],
                            leading_dim=416))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Concat,
                            bottom_names = ["reshape1", "dense"], top_names = ["concat1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Slice,
                            bottom_names = ["concat1"],
                            top_names = ["slice11", "slice12"],
                            ranges=[(0,429),(0,429)]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.MultiCross,
                            bottom_names = ["slice11"],
                            top_names = ["multicross1"],
                            num_layers=6))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["slice12"],
                            top_names = ["fc1"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc1"],
                            top_names = ["relu1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Dropout,
                            bottom_names = ["relu1"],
                            top_names = ["dropout1"],
                            dropout_rate=0.5))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["dropout1"],
                            top_names = ["fc2"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc2"],
                            top_names = ["relu2"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Dropout,
                            bottom_names = ["relu2"],
                            top_names = ["dropout2"],
                            dropout_rate=0.5))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Concat,
                            bottom_names = ["dropout2", "multicross1"],
                            top_names = ["concat2"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["concat2"],
                            top_names = ["fc3"],
                            num_output=1))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.BinaryCrossEntropyLoss,
                            bottom_names = ["fc3", "label"],
                            top_names = ["loss"]))
model.compile()
model.summary()
model.graph_to_json(graph_config_file = "dcn.json")
model.fit(max_iter = 12000, display = 500, eval_interval = 1000, snapshot = 10000, snapshot_prefix = "dcn")

Overwriting dcn_train.py


In [3]:
!python3 dcn_train.py

====================================================Model Init=====================================================
[08d06h25m16s][HUGECTR][INFO]: Global seed is 1933201219
[08d06h25m17s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-16GB
[08d06h25m17s][HUGECTR][INFO]: num of DataReader workers: 12
[08d06h25m17s][HUGECTR][INFO]: num_internal_buffers 1
[08d06h25m17s][HUGECTR][INFO]: num_internal_buffers 1
[08d06h25m17s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=1447751
===================================================Model Compile===================================================
[08d06h25m29s][HUGECTR][INFO]: gpu0 start to init embedding
[08d06h25m29s][HUGECTR][INFO]: gpu0 init embedding done
===================================================Model Summary===================================================
Label                                   Dense                         Sparse                        
label                             

<a id="23"></a>
### 2.3 Continue Training

We can continue our training based on the saved model graph, model weights and optimizer states by doing the following with Python APIs:

1. Create the solver, reader and optimizer, then initialize the model.
2. Construct the model graph from the saved JSON file.
3. Compile the model and have an overview of the model graph.
4. Load the model weights and optimizer states.
5. Fit the model, save the model weights and optimizer states implicitly.

In [4]:
!ls *.model

dcn0_opt_sparse_10000.model  dcn_dense_10000.model
dcn0_sparse_10000.model      dcn_opt_dense_10000.model


In [5]:
%%writefile dcn_continue.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 1500,
                              batchsize_eval = 4096,
                              batchsize = 4096,
                              i64_input_key = False,
                              use_mixed_precision = False,
                              repeat_dataset = True,
                              use_cuda_graph = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                                  source = ["./dcn_data/file_list.txt"],
                                  eval_source = "./dcn_data/file_list_test.txt",
                                  check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
model = hugectr.Model(solver, reader, optimizer)
model.construct_from_json(graph_config_file = "dcn.json", include_dense_network = True)
model.compile()
model.load_dense_weights("dcn_dense_10000.model")
model.load_sparse_weights(["dcn0_sparse_10000.model"])
model.load_dense_optimizer_states("dcn_opt_dense_10000.model")
model.load_sparse_optimizer_states(["dcn0_opt_sparse_10000.model"])
model.summary()
model.fit(max_iter = 5000, display = 500, eval_interval = 1000, snapshot = 10000, snapshot_prefix = "dcn")

Writing dcn_continue.py


In [6]:
!python3 dcn_continue.py

====================================================Model Init=====================================================
[08d06h30m08s][HUGECTR][INFO]: Global seed is 1391592569
[08d06h30m10s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-16GB
[08d06h30m10s][HUGECTR][INFO]: num of DataReader workers: 12
[08d06h30m10s][HUGECTR][INFO]: num_internal_buffers 1
[08d06h30m10s][HUGECTR][INFO]: num_internal_buffers 1
[08d06h30m10s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=1447751
[08d06h30m10s][HUGECTR][INFO]: Load the model graph from dcn.json, successful
===================================================Model Compile===================================================
[08d06h30m22s][HUGECTR][INFO]: gpu0 start to init embedding
[08d06h30m22s][HUGECTR][INFO]: gpu0 init embedding done
[08d06h30m22s][HUGECTR][INFO]: Loading dense model: dcn_dense_10000.model
[08d06h30m22s][HUGECTR][INFO]: Loading sparse model: dcn0_sparse_10000.model
[08d06h30m28s][HUGECTR

<a id="24"></a>
### 2.4 Inference

The HugeCTR inference is enabled by `hugectr.inference.InferenceSession.predict` method of InferenceSession, which requires dense features, embedding columns and row pointers of slots as the input and gives the prediction result as the output. We need to convert the criteo data to inference format first.

In [7]:
!python3 ../tools/criteo_predict/criteo2predict.py --src_csv_path=dcn_data/val/test.txt --src_config=../tools/criteo_predict/dcn_data.json --dst_path=./dcn_csr.txt --batch_size=1024

We can then make inference based on the saved model graph and model weights by doing the following with Python APIs:

1. Configure the inference related parameters.
2. Create the inference session.
3. Make inference with the `InferenceSession.predict` method. 

In [8]:
%%writefile dcn_inference.py
from hugectr.inference import InferenceParams, CreateInferenceSession
from mpi4py import MPI

def calculate_accuracy(labels, output):
  num_samples = len(labels)
  flags = [1 if ((labels[i] == 0 and output[i] <= 0.5) or (labels[i] == 1 and output[i] > 0.5)) else 0 for i in range(num_samples)]
  correct_samples = sum(flags)
  return float(correct_samples)/(float(num_samples)+1e-16)

data_file = open("dcn_csr.txt")
config_file = "dcn.json"
labels = [int(item) for item in data_file.readline().split(' ')]
dense_features = [float(item) for item in data_file.readline().split(' ') if item!="\n"]
embedding_columns = [int(item) for item in data_file.readline().split(' ')]
row_ptrs = [int(item) for item in data_file.readline().split(' ')]

# create parameter server, embedding cache and inference session
inference_params = InferenceParams(model_name = "dcn",
                                max_batchsize = 1024,
                                hit_rate_threshold = 0.6,
                                dense_model_file = "./dcn_dense_10000.model",
                                sparse_model_files = ["./dcn0_sparse_10000.model"],
                                device_id = 0,
                                use_gpu_embedding_cache = True,
                                cache_size_percentage = 0.9,
                                i64_input_key = False,
                                use_mixed_precision = False)
inference_session = CreateInferenceSession(config_file, inference_params)
output = inference_session.predict(dense_features, embedding_columns, row_ptrs)
accuracy = calculate_accuracy(labels, output)
print("[HUGECTR][INFO] number samples: {}, accuracy: {}".format(len(labels), accuracy))

Writing dcn_inference.py


In [9]:
!python3 dcn_inference.py

[08d06h32m29s][HUGECTR][INFO]: default_emb_vec_value is not specified using default: 0.000000
[08d06h32m32s][HUGECTR][INFO]: Global seed is 2667697857
[08d06h32m33s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
[08d06h32m33s][HUGECTR][INFO]: start create embedding for inference
[08d06h32m33s][HUGECTR][INFO]: sparse_input name data1
[08d06h32m33s][HUGECTR][INFO]: create embedding for inference success
[08d06h32m33s][HUGECTR][INFO]: Inference stage skip BinaryCrossEntropyLoss layer, replaced by Sigmoid layer
[HUGECTR][INFO] number samples: 1024, accuracy: 0.9736328125


<a id="3"></a>
## 3. Wide&Deep Demo

<a id="31"></a>
### 3.1 Download and Preprocess Data
1. Download the Kaggle Criteo dataset using the following command:
   ```shell
   $ cd ${project_root}/tools
   $ wget http://azuremlsampleexperiments.blob.core.windows.net/criteo/day_1.gz
   ```
   
   In preprocessing, we will further reduce the amounts of data to speedup the preprocessing, fill missing values, remove the feature values whose occurrences are very rare, etc. Here we choose pandas preprocessing method to make the dataset ready for HugeCTR training.

2. Preprocessing by Pandas using the following command:
   ```shell
   $ bash preprocess.sh 1 wdl_data pandas 1 1 100
   ```
   
   The first argument represents the dataset postfix. It is 1 here since day_1 is used. The second argument wdl_data is where the preprocessed data is stored. The fourth arguement (one after pandas) 1 embodies that the normalization is applied to dense features. The fifth argument 1 means that the feature crossing is applied. The last argument 100 means the number of data files in each file list.
   
3. Create a soft link to the dataset folder using the following command:
   ```shell
   $ ln ${project_root}/tools/wdl_data ${project_root}/notebooks/wdl_data
   ```

<a id="32"></a>
### 3.2 Model Oversubscription

We can train fom scratch using model oversubscriber, dump the model graph to a JSON file, and save the trained dense weights and sparse embedding weights by doing the following with Python APIs:

1. Create the solver, reader, optimizer and MOS, then initialize the model.
2. Construct the model graph by adding input, sparse embedding and dense layers in order.
3. Compile the model and have an overview of the model graph.
4. Dump the model graph to the JSON file.
5. Fit the model.
6. Save the model weights and optimizer states explicitly.

To use model oversubscription, we should specify `repeat_dataset` as False. The data in each file list will be trained for specified number of epochs under this mode.

In [10]:
%%writefile wdl_train.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 5000,
                              batchsize_eval = 1024,
                              batchsize = 1024,
                              vvgpu = [[0]],
                              i64_input_key = False,
                              use_mixed_precision = False,
                              repeat_dataset = False,
                              use_cuda_graph = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                          source = ["wdl_data/file_list."+str(i)+".txt" for i in range(2)],
                          keyset = ["wdl_data/file_list."+str(i)+".keyset" for i in range(2)],
                          eval_source = "wdl_data/file_list.2.txt",
                          check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
mos = hugectr.CreateMOS(train_from_scratch = True, use_host_memory_ps = True, dest_sparse_models = ["./wdl_0_sparse_model", "./wdl_1_sparse_model"])
model = hugectr.Model(solver, reader, optimizer, mos)
model.add(hugectr.Input(label_dim = 1, label_name = "label",
                        dense_dim = 13, dense_name = "dense",
                        data_reader_sparse_param_array = 
                        [hugectr.DataReaderSparseParam(hugectr.DataReaderSparse_t.Distributed, 30, 2, 1),
                        hugectr.DataReaderSparseParam(hugectr.DataReaderSparse_t.Distributed, 30, 1, 26)],
                        sparse_names = ["wide_data", "deep_data"]))
model.add(hugectr.SparseEmbedding(embedding_type = hugectr.Embedding_t.DistributedSlotSparseEmbeddingHash, 
                            max_vocabulary_size_per_gpu = 5863985,
                            embedding_vec_size = 1,
                            combiner = 0,
                            sparse_embedding_name = "sparse_embedding2",
                            bottom_name = "wide_data",
                            optimizer = optimizer))
model.add(hugectr.SparseEmbedding(embedding_type = hugectr.Embedding_t.DistributedSlotSparseEmbeddingHash, 
                            max_vocabulary_size_per_gpu = 5863985,
                            embedding_vec_size = 16,
                            combiner = 0,
                            sparse_embedding_name = "sparse_embedding1",
                            bottom_name = "deep_data",
                            optimizer = optimizer))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Reshape,
                            bottom_names = ["sparse_embedding1"],
                            top_names = ["reshape1"],
                            leading_dim=416))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Reshape,
                            bottom_names = ["sparse_embedding2"],
                            top_names = ["reshape2"],
                            leading_dim=1))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Concat,
                            bottom_names = ["reshape1", "dense"], top_names = ["concat1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["concat1"],
                            top_names = ["fc1"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc1"],
                            top_names = ["relu1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Dropout,
                            bottom_names = ["relu1"],
                            top_names = ["dropout1"],
                            dropout_rate=0.5))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["dropout1"],
                            top_names = ["fc2"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc2"],
                            top_names = ["relu2"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Dropout,
                            bottom_names = ["relu2"],
                            top_names = ["dropout2"],
                            dropout_rate=0.5))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["dropout2"],
                            top_names = ["fc3"],
                            num_output=1))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Add,
                            bottom_names = ["fc3", "reshape2"],
                            top_names = ["add1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.BinaryCrossEntropyLoss,
                            bottom_names = ["add1", "label"],
                            top_names = ["loss"]))
model.compile()
model.summary()
model.graph_to_json(graph_config_file = "wdl.json")
model.fit(num_epochs = 1, display = 500, eval_interval = 1000)
model.set_source(source = ["wdl_data/file_list.3.txt", "wdl_data/file_list.4.txt"], keyset = ["wdl_data/file_list.3.keyset", "wdl_data/file_list.4.keyset"], eval_source = "wdl_data/file_list.5.txt")
model.fit(num_epochs = 1, display = 500, eval_interval = 1000)
model.save_params_to_files("wdl_mos")

Overwriting wdl_train.py


In [11]:
!python3 wdl_train.py

[1622631939.946016] [dgx1v-loki-29:947  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_0) failed: No such file or directory
[1622631939.946177] [dgx1v-loki-29:947  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_1) failed: No such file or directory
[1622631939.946294] [dgx1v-loki-29:947  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_2) failed: No such file or directory
[1622631939.946401] [dgx1v-loki-29:947  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_3) failed: No such file or directory
====================================================Model Init=====================================================
[02d11h05m39s][HUGECTR][INFO]: Global seed is 1070170292
[02d11h05m41s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-32GB
[02d11h05m41s][HUGECTR][INFO]: num of DataReader workers: 12
[02d11h05m41s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h05m41s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h05m4

<a id="33"></a>
### 3.3 Fine-tune

We can only load the sparse embedding layers their corresponding weights, and then construct a new dense network. The dense weights will be trained first and the sparse weights will be fine-tuned later. We can achieve this by doing the following with Python APIs:

1. Create the solver, reader and optimizer, then initialize the model.
2. Load the sparse embedding layers from the saved JSON file.
3. Add the dense layers on top of the loaded model graph.
4. Compile the model and have an overview of the model graph.
5. Load the sparse weights and freeze the sparse embedding layers.
6. Train the dense weights.
7. Unfreeze the sparse embedding layers and freeze the dense layers, reset the learning rate scheduler with a small rate.
8. Fine-tune the sparse weights.

In [12]:
%%writefile wdl_fine_tune.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 5000,
                              batchsize_eval = 1024,
                              batchsize = 1024,
                              vvgpu = [[0]],
                              i64_input_key = False,
                              use_mixed_precision = False,
                              repeat_dataset = False,
                              use_cuda_graph = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                          source = ["wdl_data/file_list.6.txt"],
                          eval_source = "wdl_data/file_list.7.txt",
                          check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
model = hugectr.Model(solver, reader, optimizer)
model.construct_from_json(graph_config_file = "wdl.json", include_dense_network = False)
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Reshape,
                            bottom_names = ["sparse_embedding1"],
                            top_names = ["reshape1"],
                            leading_dim=416))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Reshape,
                            bottom_names = ["sparse_embedding2"],
                            top_names = ["reshape2"],
                            leading_dim=1))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Concat,
                            bottom_names = ["reshape1", "reshape2", "dense"], top_names = ["concat1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["concat1"],
                            top_names = ["fc1"],
                            num_output=1024))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.ReLU,
                            bottom_names = ["fc1"],
                            top_names = ["relu1"]))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.Dropout,
                            bottom_names = ["relu1"],
                            top_names = ["dropout1"],
                            dropout_rate=0.5))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.InnerProduct,
                            bottom_names = ["dropout1"],
                            top_names = ["fc2"],
                            num_output=1))
model.add(hugectr.DenseLayer(layer_type = hugectr.Layer_t.BinaryCrossEntropyLoss,
                            bottom_names = ["fc2", "label"],
                            top_names = ["loss"]))
model.compile()
model.summary()
model.load_sparse_weights(["wdl_0_sparse_model", "wdl_1_sparse_model"])
model.freeze_embedding()
model.fit(num_epochs = 1, display = 500, eval_interval = 1000, snapshot = 100000, snapshot_prefix = "wdl")
model.unfreeze_embedding()
model.freeze_dense()
model.reset_learning_rate_scheduler(base_lr = 0.0001)
model.fit(num_epochs = 2, display = 500, eval_interval = 1000, snapshot = 100000, snapshot_prefix = "wdl")

Overwriting wdl_fine_tune.py


In [13]:
!python3 wdl_fine_tune.py

[1622633077.070609] [dgx1v-loki-23:856  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_0) failed: No such file or directory
[1622633077.070754] [dgx1v-loki-23:856  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_1) failed: No such file or directory
[1622633077.070884] [dgx1v-loki-23:856  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_2) failed: No such file or directory
[1622633077.071006] [dgx1v-loki-23:856  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_3) failed: No such file or directory
====================================================Model Init=====================================================
[02d11h24m37s][HUGECTR][INFO]: Global seed is 1386045165
[02d11h24m39s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-32GB
[02d11h24m39s][HUGECTR][INFO]: num of DataReader workers: 12
[02d11h24m39s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h24m39s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h24m3

<a id="34"></a>
### 3.4 Low-level Training

The low-level training APIs are maintained in the enhanced HugeCTR Python interface. If you want to have precise control of each training iteration and each evaluation step, you may find it helpful to use these APIs. Since the data reader behavior is different in epoch mode and non-epoch mode, we should pay attention to how to tweak the data reader when using low-level training. We will denmonstrate how to write the low-level training scripts for non-epoch mode, epoch mode and model oversubscription mode.

In [14]:
%%writefile wdl_non_epoch.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 5000,
                              batchsize_eval = 1024,
                              batchsize = 1024,
                              vvgpu = [[0]],
                              i64_input_key = False,
                              use_mixed_precision = False,
                              repeat_dataset = True,
                              use_cuda_graph = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                          source = ["wdl_data/file_list.0.txt"],
                          eval_source = "wdl_data/file_list.1.txt",
                          check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
model = hugectr.Model(solver, reader, optimizer)
model.construct_from_json(graph_config_file = "wdl.json", include_dense_network = True)
model.compile()
model.start_data_reading()
lr_sch = model.get_learning_rate_scheduler()
max_iter = 2000
for i in range(max_iter):
  lr = lr_sch.get_next()
  model.set_learning_rate(lr)
  model.train()
  if (i%100 == 0):
    loss = model.get_current_loss()
    print("[HUGECTR][INFO] iter: {}; loss: {}".format(i, loss))
  if (i%1000 == 0 and i != 0):
    for _ in range(solver.max_eval_batches):
      model.eval()
    metrics = model.get_eval_metrics()
    print("[HUGECTR][INFO] iter: {}, {}".format(i, metrics))
model.save_params_to_files("./", max_iter)

Writing wdl_non_epoch.py


In [15]:
!python3 wdl_non_epoch.py

[1622633305.919863] [dgx1v-loki-23:898  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_0) failed: No such file or directory
[1622633305.919988] [dgx1v-loki-23:898  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_1) failed: No such file or directory
[1622633305.920087] [dgx1v-loki-23:898  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_2) failed: No such file or directory
[1622633305.920174] [dgx1v-loki-23:898  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_3) failed: No such file or directory
====================================================Model Init=====================================================
[02d11h28m25s][HUGECTR][INFO]: Global seed is 920733150
[02d11h28m26s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-32GB
[02d11h28m26s][HUGECTR][INFO]: num of DataReader workers: 12
[02d11h28m27s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h28m27s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h28m27

In [16]:
%%writefile wdl_epoch.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 5000,
                              batchsize_eval = 1024,
                              batchsize = 1024,
                              vvgpu = [[0]],
                              i64_input_key = False,
                              use_mixed_precision = False,
                              repeat_dataset = False,
                              use_cuda_graph = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                          source = ["wdl_data/file_list.0.txt"],
                          eval_source = "wdl_data/file_list.1.txt",
                          check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
model = hugectr.Model(solver, reader, optimizer)
model.construct_from_json(graph_config_file = "wdl.json", include_dense_network = True)
model.compile()
lr_sch = model.get_learning_rate_scheduler()
data_reader_train = model.get_data_reader_train()
data_reader_eval = model.get_data_reader_eval()
data_reader_eval.set_source()
data_reader_eval_flag = True
iteration = 0
for epoch in range(2):
  print("[HUGECTR][INFO] epoch: ", epoch)
  data_reader_train.set_source()
  data_reader_train_flag = True
  while True:
    lr = lr_sch.get_next()
    model.set_learning_rate(lr)
    data_reader_train_flag = model.train()
    if not data_reader_train_flag:
      break
    if iteration % 1000 == 0:
      batches = 0
      while data_reader_eval_flag:
        if batches >= solver.max_eval_batches:
          break
        data_reader_eval_flag = model.eval()
        batches += 1
      if not data_reader_eval_flag:
        data_reader_eval.set_source()
        data_reader_eval_flag = True
      metrics = model.get_eval_metrics()
      print("[HUGECTR][INFO] iter: {}, metrics: {}".format(iteration, metrics))
    iteration += 1
model.save_params_to_files("./", iteration)

Writing wdl_epoch.py


In [17]:
!python3 wdl_epoch.py

[1622633343.670005] [dgx1v-loki-23:940  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_0) failed: No such file or directory
[1622633343.670133] [dgx1v-loki-23:940  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_1) failed: No such file or directory
[1622633343.670230] [dgx1v-loki-23:940  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_2) failed: No such file or directory
[1622633343.670319] [dgx1v-loki-23:940  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_3) failed: No such file or directory
====================================================Model Init=====================================================
[02d11h29m03s][HUGECTR][INFO]: Global seed is 3482891501
[02d11h29m04s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-32GB
[02d11h29m04s][HUGECTR][INFO]: num of DataReader workers: 12
[02d11h29m04s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h29m04s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h29m0

In [18]:
%%writefile wdl_mos.py
import hugectr
from mpi4py import MPI
solver = hugectr.CreateSolver(max_eval_batches = 5000,
                              batchsize_eval = 1024,
                              batchsize = 1024,
                              vvgpu = [[0]],
                              i64_input_key = False,
                              use_mixed_precision = False,
                              repeat_dataset = False,
                              use_cuda_graph = True)
reader = hugectr.DataReaderParams(data_reader_type = hugectr.DataReaderType_t.Norm,
                          source = ["wdl_data/file_list."+str(i)+".txt" for i in range(2)],
                          keyset = ["wdl_data/file_list."+str(i)+".keyset" for i in range(2)],
                          eval_source = "wdl_data/file_list.2.txt",
                          check_type = hugectr.Check_t.Sum)
optimizer = hugectr.CreateOptimizer(optimizer_type = hugectr.Optimizer_t.Adam)
mos = hugectr.CreateMOS(train_from_scratch = True, use_host_memory_ps = True, dest_sparse_models = ["./wdl_low_0_sparse_model", "./wdl_low_1_sparse_model"])
model = hugectr.Model(solver, reader, optimizer, mos)
model.construct_from_json(graph_config_file = "wdl.json", include_dense_network = True)
model.compile()
lr_sch = model.get_learning_rate_scheduler()
data_reader_train = model.get_data_reader_train()
data_reader_eval = model.get_data_reader_eval()
model_oversubscriber = model.get_model_oversubscriber()
dataset = [("wdl_data/file_list."+str(i)+".txt", "wdl_data/file_list."+str(i)+".keyset") for i in range(2)]
data_reader_eval.set_source("wdl_data/file_list.2.txt")
data_reader_eval_flag = True
iteration = 0
for file_list, keyset_file in dataset:
  data_reader_train.set_source(file_list)
  data_reader_train_flag = True
  model_oversubscriber.update(keyset_file)
  while True:
    lr = lr_sch.get_next()
    model.set_learning_rate(lr)
    data_reader_train_flag = model.train()
    if not data_reader_train_flag:
      break
    if iteration % 1000 == 0:
      batches = 0
      while data_reader_eval_flag:
        if batches >= solver.max_eval_batches:
          break
        data_reader_eval_flag = model.eval()
        batches += 1
      if not data_reader_eval_flag:
        data_reader_eval.set_source()
        data_reader_eval_flag = True
      metrics = model.get_eval_metrics()
      print("[HUGECTR][INFO] iter: {}, metrics: {}".format(iteration, metrics))
    iteration += 1
  print("[HUGECTR][INFO] trained with data in {}".format(file_list))
model.save_params_to_files("wdl_mos")

Overwriting wdl_mos.py


In [19]:
!python3 wdl_mos.py

[1622633460.777343] [dgx1v-loki-23:982  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_0) failed: No such file or directory
[1622633460.777472] [dgx1v-loki-23:982  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_1) failed: No such file or directory
[1622633460.777568] [dgx1v-loki-23:982  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_2) failed: No such file or directory
[1622633460.777657] [dgx1v-loki-23:982  :0]          ib_md.c:1678 UCX  ERROR ibv_open_device(mlx5_3) failed: No such file or directory
====================================================Model Init=====================================================
[02d11h31m00s][HUGECTR][INFO]: Global seed is 2434655721
[02d11h31m01s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-32GB
[02d11h31m01s][HUGECTR][INFO]: num of DataReader workers: 12
[02d11h31m01s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h31m01s][HUGECTR][INFO]: num_internal_buffers 1
[02d11h31m0